In [1]:
import taxcalc as tc
from pathlib import Path
import os
import numpy as np

In [2]:
# Policy1:  
# Base EITC on individual earnings, EITC only available for tax units with <$100,000
# Note: "reformed policy with limitation on EITC with income celling ~ income tax revenue" is result of Policy1

In [3]:
# define the calculator object with data file and policy
recs = tc.Records(data = 'puf.csv')
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [4]:
# reform2: combine effect
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [5]:
totalam = 0
# Policy1: switch EITC from household to individuals & celling of $100,000 income on tax units
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)
    calc2.advance_to_year(year)
    calc2.calc_all()
    
    # calculate the difference of total EITC between the limited (household with income lower than $100,000) and normal, for reform2
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['c00100'] < 100000]
    
    vardf = tc.add_income_table_row_variable(vardf, 'c00100', tc.SOI_AGI_BINS)
    gbydf = vardf.groupby('table_row', as_index=False)
    
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    tot_amount = 0.
    for grp_interval, grp in gbydf:
        amount = (grp['eitc'] * grp['s006']).sum()
        tot_amount += amount

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount

    eitc_diff = ltot_amount - tot_amount
    itax_rev2 = calc2.weighted_total('iitax')
    

    # CHANGE 
    litax = itax_rev2 + tot_amount - ltot_amount 
    # litax = itax_rev2 + eitc_diff
    newcost = litax - itax_rev0
    totalam = totalam + newcost
    print("reformed policy with indiv switch & income celling ~ income tax revenue: ", litax)
    print("difference: reform vs current law ", litax - itax_rev0 )



year:  2023
current law income tax revenue:  2155167870079.055


/opt/miniconda3/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


reformed policy with indiv switch & income celling ~ income tax revenue:  2141953378407.464
difference: reform vs current law  -13214491671.59082
year:  2024
current law income tax revenue:  2205120354993.102
reformed policy with indiv switch & income celling ~ income tax revenue:  2191367606897.1548
difference: reform vs current law  -13752748095.947266
year:  2025
current law income tax revenue:  2323375424356.6626
reformed policy with indiv switch & income celling ~ income tax revenue:  2309178593591.558
difference: reform vs current law  -14196830765.104492
year:  2026
current law income tax revenue:  2650040927443.353
reformed policy with indiv switch & income celling ~ income tax revenue:  2635441657701.075
difference: reform vs current law  -14599269742.277832
year:  2027
current law income tax revenue:  2772128822161.977
reformed policy with indiv switch & income celling ~ income tax revenue:  2757200516299.652
difference: reform vs current law  -14928305862.325195
year:  2028


In [6]:
# Policy2: 
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# eliminate EITC for single childless adults (BUT NOT FOR MARRIED CHILDLESS ADULTS)

In [7]:
# define the calculator object with data file and policy
recs = tc.Records(data = 'puf.csv')
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [8]:
# reform2: swich to indiv
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)


In [9]:
# reform4: only calculate the single EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 3733.0, 6164.0, 6935.0]}}
             
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [10]:
totalam = 0
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate single only eitc amount
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select single only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 1].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate non-single eitc
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['MARS'] != 1].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount


    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 

    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    # print("reform 4 single eitc ", fltot_amount )
    # print("reform 2 non single eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
#print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    


year:  2023
current law income tax revenue:  2155167870079.055
reform:  2143754638560.5994
difference of income revenue: reform vs current law -11413231518.455566
difference of eitc: reform vs current law  11413078134.120346
year:  2024
current law income tax revenue:  2205120354993.102
reform:  2193276715711.281
difference of income revenue: reform vs current law -11843639281.821045
difference of eitc: reform vs current law  11843519140.253342
year:  2025
current law income tax revenue:  2323375424356.6626
reform:  2311121992774.151
difference of income revenue: reform vs current law -12253431582.511719
difference of eitc: reform vs current law  12253340995.235886
year:  2026
current law income tax revenue:  2650040927443.353
reform:  2637391224144.549
difference of income revenue: reform vs current law -12649703298.8042
difference of eitc: reform vs current law  12649412080.361588
year:  2027
current law income tax revenue:  2772128822161.977
reform:  2759166902222.1104
difference of

In [11]:
# Policy3
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# replace the married parent EITC schedules with the “super-schedule” for married couples from Winship 2021 paper

In [12]:
# define the calculator object with data file and policy
recs = tc.Records(data = 'puf.csv')
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [13]:
# reform2: combine effect
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [14]:
# reform4: swich + super schedule -- only calculate the married EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 6935.0, 6935.0, 6935.0]},
           "EITC_rt": {"2022": [0.45, 0.45, 0.45, 0.45]},
           "EITC_prt": {"2022": [0.24, 0.24, 0.24, 0.24]},
           "EITC_ps": {"2022": [20130.0, 20130.0, 20130.0, 20130.0]},
           "EITC_ps_MarriedJ": {"2022": [18390.0, 18390.0, 18390.0, 18390.0]}
             }
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [15]:
totalam = 0
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate married only eitc amount
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select married only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 2].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate single eitc
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # CHECK THIS LINE fvardf??????????????????
    lvardf = vardf.loc[vardf['MARS'] != 2].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount
        

    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 

    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    # print("reform 4 married eitc ", fltot_amount )
    # print("reform 2 non married eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    

year:  2023
current law income tax revenue:  2155167870079.055
reform:  2131165083377.5078
difference of income revenue: reform vs current law -24002786701.54712
difference of eitc: reform vs current law  24002633317.21187
year:  2024
current law income tax revenue:  2205120354993.102
reform:  2180063101505.1375
difference of income revenue: reform vs current law -25057253487.9646
difference of eitc: reform vs current law  25057133346.397537
year:  2025
current law income tax revenue:  2323375424356.6626
reform:  2297344387025.2334
difference of income revenue: reform vs current law -26031037331.4292
difference of eitc: reform vs current law  26030946744.153183
year:  2026
current law income tax revenue:  2650040927443.353
reform:  2623099900742.2275
difference of income revenue: reform vs current law -26941026701.12549
difference of eitc: reform vs current law  26940735482.682587
year:  2027
current law income tax revenue:  2772128822161.977
reform:  2744385098075.411
difference of in

In [16]:
# Policy4
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# eliminate EITC for single childless adults (BUT NOT FOR MARRIED CHILDLESS ADULTS),
# replace the married parent EITC schedules with the “super-schedule” for married couples from Winship 2021 paper

In [17]:
# define the calculator object with data file and policy
recs = tc.Records(data = 'puf.csv')
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [18]:
# reform1: switch only
reform1 = {"EITC_indiv": {"2022": True}}
pol1 = tc.Policy()
pol1.implement_reform(reform1, print_warnings=False, raise_errors=False)
calc1 = tc.Calculator(policy=pol1, records=recs)

In [19]:
# reform2: only calculate the single EITC
reform2 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 3733.0, 6164.0, 6935.0]}}
             
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [20]:
# reform4: swich + super schedule -- only calculate the married EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 6935.0, 6935.0, 6935.0]},
           "EITC_rt": {"2022": [0.45, 0.45, 0.45, 0.45]},
           "EITC_prt": {"2022": [0.24, 0.24, 0.24, 0.24]},
           "EITC_ps": {"2022": [20130.0, 20130.0, 20130.0, 20130.0]},
           "EITC_ps_MarriedJ": {"2022": [18390.0, 18390.0, 18390.0, 18390.0]}
             }
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [ ]:
totalam = 0

for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate married only eitc amount -- super schedule
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select married only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 2].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate single eitc -- zero out childless
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # CHECK THIS LINE !!!!!!!
    lvardf = vardf.loc[vardf['MARS'] == 1].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount
        

    calc1.advance_to_year(year)
    calc1.calc_all()
    nvardf = calc1.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # mars is 3 or 4 or 5
    nlvardf = nvardf.loc[(nvardf['MARS'] == 3) | (nvardf['MARS'] == 4) | (nvardf['MARS'] == 5)].loc[nvardf['c00100'] < 100000]
    nlvardf = tc.add_income_table_row_variable(nlvardf, 'c00100', tc.SOI_AGI_BINS)
    nlgbydf = nlvardf.groupby('table_row', as_index=False)

    nltot_amount = 0.
    for lgrp_interval, lgrp in nlgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        nltot_amount += lamount    
        
        
        
        
        
    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount + nltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 


    # print("reform 4 married eitc ", fltot_amount )
    # print("reform 2 non married eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    

year:  2023
current law income tax revenue:  2155167870079.055
reform:  2132968364467.713
difference of income revenue: reform vs current law -22199505611.34204
difference of eitc: reform vs current law  22199352227.00682
year:  2024
current law income tax revenue:  2205120354993.102
reform:  2181974299633.4714
difference of income revenue: reform vs current law -23146055359.630615
difference of eitc: reform vs current law  23145935218.063583
year:  2025
current law income tax revenue:  2323375424356.6626
reform:  2299289935961.7593
difference of income revenue: reform vs current law -24085488394.90332
difference of eitc: reform vs current law  24085397807.62764
year:  2026
current law income tax revenue:  2650040927443.353
reform:  2625051953388.475
difference of income revenue: reform vs current law -24988974054.87793
difference of eitc: reform vs current law  24988682836.43521
year:  2027
current law income tax revenue:  2772128822161.977
reform:  2746354210521.103
difference of inc